In [189]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [190]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Очищення датасетів

### Аналізуючи датасет можна викинути такі колонки: PassengerId, Name, Ticket, Cabin

Cabin - я видалив, тому що було відсутньо близько 77% всіх значень

In [191]:
train_data = train_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'])
test_data = test_data.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'])

# Заміняю пусті значення для колонки Age на ті, що попадаються найчастіше
train_data = train_data.fillna({'Age': train_data['Age'].mode()[0]})
test_data = test_data.fillna({'Age': train_data['Age'].mode()[0]})

# # Заміняю пусті значення для колонки Embarked на ті, що попадаються найчастіше
# train_data = train_data.fillna({'Embarked': train_data['Embarked'].mode()[0]})
# test_data = test_data.fillna({'Embarked': train_data['Embarked'].mode()[0]})

test_data = test_data.fillna({'Fare': train_data['Fare'].mode()[0]})

train_data = train_data.replace(['male','female'], [0,1])
test_data = test_data.replace(['male', 'female'], [0,1])

y = train_data["Survived"]
train_data = train_data.drop(columns=["Survived"])

X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.3, random_state=1)

y_test


862    1
223    0
84     1
680    0
535    1
      ..
247    1
551    0
239    0
484    1
92     0
Name: Survived, Length: 268, dtype: int64

# Спершу тренуватиму на RandomForest

100 - дерев, глибиною в 5

In [192]:
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

# output = pd.DataFrame({'PassengerId': X_test['PassengerId'], 'Survived': predictions})

# output

0.7873134328358209
